In [ ]:
!pip install PyMuPDF -qqq
!pip install llama-cpp-python -qqq
!pip install langchain langchain_huggingface -qqq
!pip install -U qdrant-client -qqq
!pip install gradio -qqq

In [34]:
import os
import re
import fitz
import shutil
from typing import List, Tuple, Dict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sentence_transformers import SentenceTransformer, CrossEncoder
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    VectorParams,
    Distance,
    PointStruct
)
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from llama_cpp import Llama
from huggingface_hub import login
import gradio as gr

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("gemma_3")
login(token=secret_value_0)

In [7]:
class EmbeddingManager:
    def __init__(self, model_name="ibm-granite/granite-embedding-english-r2", device='cuda:0'):
        self.model_name = model_name
        self.device = device
        self.model = None
        self._initialize_model()

    def _initialize_model(self):
        try:
            self.model = SentenceTransformer(
                self.model_name,
                trust_remote_code=True,
                device=self.device
            )
            self.tokenizer = self.model.tokenizer
            print(f"Embedding Model '{self.model_name}' Initialized on {self.device}")
        except Exception as e:
            print("Model Not Found:", e)

    def get_embedding(self, txt:List[str]):
        if not self.model:
            raise ValueError("Model not loaded")
            
        print(f"Generating embeddings for {len(txt)} texts ... ")
        embeddings = self.model.encode(txt, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

    def get_embedding_dim(self):
        dim = self.model.get_sentence_embedding_dimension()
        print("Model Dimension:", dim)

embedding_model = EmbeddingManager()
embedding_model.get_embedding_dim()

Embedding Model 'ibm-granite/granite-embedding-english-r2' Initialized on cuda:0
Model Dimension: 768


In [8]:
topic_regex = re.compile(r'(\d{3})\s+[A-Z]+')
topic_dict = []

doc = fitz.open("/kaggle/input/diy-book/DIY_BOOK.pdf")
full_text = ""

for page in doc:
    full_text += page.get_text()

splits = list(topic_regex.finditer(full_text))
if splits:
    preamble = full_text[:splits[0].start()].strip()
    for i, match in enumerate(splits):
        topic_num = match.group(1)
        topic_start = match.start()
        topic_end = splits[i+1].start() if i+1 < len(splits) else len(full_text)
        topic_content = full_text[topic_start:topic_end].strip()
        topic_match = re.search(r'^(\d{3}\s+[A-Z ]+)\s*(.*)', topic_content, re.DOTALL)
        if topic_match:
            topic_name = topic_match.group(1)
            content = topic_match.group(2)
            topic_name_without_num = re.sub(r'^\d{3}\s*', '', topic_name).strip()
        else:
            topic_name_without_num = ""
            content = topic_content
        if len(content)>50:
            topic_dict.append(
                {
                    'topic_no': int(topic_num),
                    'topic_name': topic_name_without_num,
                    'content': content
                }
            )
else:
    topic_dict.append({'topic_no': None, 'topic_name': 'FULL_TEXT', 'content': full_text.strip()})

topic_dict = topic_dict[1:]

In [9]:
class PreprocessPdf:
    def __init__(self, model, content_list: List[Dict]):
        self.model = model
        self.content_list = content_list
        self.text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer=self.model,
            chunk_size=512,
            chunk_overlap=50,
            separators=["."]
        )

    def clean_text(self,txt:str):
        txt = txt.replace('\n',' ')
        return txt

    def process_content(self) -> List[Dict]:
        all_chunks_metadata = []
        
        for topic_item in self.content_list:
            topic_no = topic_item.get('topic_no')
            topic_name = topic_item.get('topic_name') 
            full_text = topic_item.get('content', '') 
    
            if not full_text:
                continue

            chunks = self.text_splitter.split_text(full_text)

            for idx, chunk in enumerate(chunks):
                chunk = self.clean_text(chunk)
                chunk_token_count = len(self.model(chunk)['input_ids'])
                all_chunks_metadata.append({
                    'topic_no': topic_no,
                    'chunk_no': idx,
                    'topic': topic_name,
                    'chunk_char_count': len(chunk),
                    'chunk_word_count': len(chunk.split()),
                    'chunk_token_count': chunk_token_count,
                    'text': chunk.lower()
                })
        
        return all_chunks_metadata


pp = PreprocessPdf(embedding_model.tokenizer,topic_dict)
metadata = pp.process_content()

In [10]:
len(metadata)

378

In [11]:
metadata[:2]

[{'topic_no': 1,
  'chunk_no': 0,
  'topic': 'BUILD A BASIC TOOLBOX',
  'chunk_char_count': 2130,
  'chunk_word_count': 346,
  'chunk_token_count': 475,
  'text': 'from simple fix-ups to major home improvements, the right tools make all the difference. novice diyers should assemble a toolbox with a handful of essential tools that are most widely applicable to any job. additional materials can be purchased as required, expanding your arsenal as your experience and skill set improve. choose a toolbox with ample storage space for large, irregular items such as a hammer and square, but one that’s light and compact enough for easy portability.  hammer from hanging a picture frame to building a wall, the right hammer is an essential household tool. you’ll find specialized models for heavy-duty framing and for fine woodwork, but a good basic model should be medium weight (16 to 20 ounces) with a smooth-faced head, rip claw, and non-wood handle.  screwdrivers whether you’re putting something t

In [12]:
df = pd.DataFrame(metadata)

In [13]:
df.sample(5)

,topic_no,chunk_no,topic,chunk_char_count,chunk_word_count,chunk_token_count,text
125,113,0,UPDATE FIXTURES,2201,383,465,your primary concern when you are selecting a ...
222,197,0,GET A COMPLEX,2079,337,415,crown molding and casing are both available in...
196,174,0,REPAIR PLASTER,713,127,157,"for plaster repairs, you can use a powder-base..."
19,16,0,GET SOME AIR,989,160,210,air power is great way to take your home shop ...
176,157,0,PERFORM SURGERY,1053,174,224,"in some cases, the best solution may be to cut..."


In [14]:
df.describe()

,topic_no,chunk_no,chunk_char_count,chunk_word_count,chunk_token_count
count,378.000000,378.000000,378.000000,378.000000,378.000000
mean,166.378307,0.164021,1144.402116,195.296296,253.902116
std,94.615134,0.573117,684.830705,108.101887,144.256250
min,1.000000,0.000000,76.000000,15.000000,28.000000
25%,87.000000,0.000000,635.750000,109.750000,141.000000
50%,168.000000,0.000000,1016.500000,174.000000,226.500000
75%,246.750000,0.000000,1572.500000,275.750000,356.500000
max,321.000000,6.000000,7605.000000,987.000000,1468.000000


In [15]:
class VectorStore:
    def __init__(self, db_name: str, db_path: str, embedding_model):
        self.db_name = db_name
        self.embedding_model = embedding_model
        self.db_path = db_path  
        self.client = None
        self._initialize_vector_store()

    def _initialize_vector_store(self):
        try:
            self.client = QdrantClient(path=self.db_path)
            
            print(f"Vector Store initialized successfully at path: '{self.db_path}'")
        except Exception as e:
            print(f"Vector Store initialization failed: {e}")

    def create_store(self, dim=768):
        if not self.client:
            print("Client not initialized. Cannot create store.")
            return
        try:
            if self.client.collection_exists(self.db_name):
                self.client.delete_collection(self.db_name)
                print(f"Dropped existing collection: '{self.db_name}'")

            self.client.create_collection(
                collection_name=self.db_name,
                vectors_config=VectorParams(size=dim, distance=Distance.COSINE)
            )
            
            print(f"Vector Store Collection '{self.db_name}' Created Successfully with dimension {dim}")

        except Exception as e:
            print(f"Vector Store Creation Failed: {e}")

    def add_documents(self, data_list: List[Dict]):
        if not self.client:
            print("No Client found. Cannot add documents.")
            return
        try:
            texts = [d['text'] for d in data_list]
            embeddings = self.embedding_model.get_embedding(texts)
            if embeddings is None:
                print("Embedding generation failed. Aborting document addition.")
                return

            points = []
            for idx, d in enumerate(data_list):
                points.append(
                    PointStruct(
                        id=str(uuid.uuid4()),  
                        vector=embeddings[idx].tolist(),  
                        payload={
                            "topic_no": d.get("topic_no"),
                            "chunk_no": d.get("chunk_no"),
                            "topic": d.get("topic"),
                            "chunk_char_count": d.get("chunk_char_count"),
                            "chunk_word_count": d.get("chunk_word_count"),
                            "chunk_token_count": d.get("chunk_token_count"),
                            "text": d.get("text")
                        }
                    )
                )
            
            if not points:
                print("No documents to add.")
                return

            self.client.upsert(collection_name=self.db_name, points=points, wait=True)
            print(f"{len(points)} documents added to Qdrant Vector Store")
        except Exception as e:
            print(f"Error adding documents: {e}")

In [16]:
DB_STORAGE_PATH = "/kaggle/working/vectordb" 
COLLECTION_NAME = "KnowDIY_DB"

if os.path.exists(DB_STORAGE_PATH):
    print(f"Found existing database path. Deleting '{DB_STORAGE_PATH}'...")
    try:
        shutil.rmtree(DB_STORAGE_PATH)
        print("Successfully cleaned old database directory.")
    except Exception as e:
        print(f"Error cleaning directory: {e}")
else:
    print(f"Directory '{DB_STORAGE_PATH}' not found. No cleaning needed.")

Directory '/kaggle/working/vectordb' not found. No cleaning needed.


In [17]:
vs = VectorStore(COLLECTION_NAME, DB_STORAGE_PATH, embedding_model)
vs.create_store(dim=768)
vs.add_documents(metadata)

Vector Store initialized successfully at path: '/kaggle/working/vectordb'
Vector Store Collection 'KnowDIY_DB' Created Successfully with dimension 768
Generating embeddings for 378 texts ... 


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

W1019 10:04:40.765000 37 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Generated embeddings with shape: (378, 768)
378 documents added to Qdrant Vector Store


In [18]:
class Retriever:
    def __init__(self, emb_model: EmbeddingManager, vectordb: VectorStore):
        self.emb_model = emb_model
        self.vectordb = vectordb

    def retrieve(self, query: str, top_k: int = 5, threshold: float = 0.3):
        query_emb = self.emb_model.get_embedding([query])

        out = self.vectordb.client.query_points(
            collection_name=self.vectordb.db_name,
            query=query_emb[0],  
            limit=top_k,
            with_payload=True
        )
        
        texts = [o.payload['text'] for o in out.points if o.score > threshold]
        return texts

retriever = Retriever(emb_model=embedding_model, vectordb=vs)

In [20]:
class ReRanker:
    def __init__(self, model_name="ibm-granite/granite-embedding-reranker-english-r2", device='cuda:0'):
        self.model_name = model_name
        self.device = device
        self._initialize_model()

    def _initialize_model(self):
        try:
            self.model = CrossEncoder(
                self.model_name,
                trust_remote_code=True,
                device=self.device
            )
            print(f"ReRanker Model '{self.model_name}' initialized on {self.device}")
        except Exception as e:
            print("Model Not Found:", e)

    def rerank(self, query: str, passages: List[str]):
        pairs = [(query, passage) for passage in passages]
        scores = self.model.predict(pairs)
        ranked_indices = np.argsort(scores)[::-1]
        ranked_passages = [passages[i] for i in ranked_indices]
        return ranked_passages, scores[ranked_indices]

    def get_embedding_dim(self):
        dim = self.model.config.hidden_size
        print("Model Dimension:", dim)

reranker = ReRanker()
reranker.get_embedding_dim()

ReRanker Model 'ibm-granite/granite-embedding-reranker-english-r2' initialized on cuda:0
Model Dimension: 768


In [21]:
PROMPT_TEMPLATE = """You are an expert DIY assistant. Your tone is helpful, confident, and you give clear, direct answers.

Your task is to answer the user's question clearly, confidently, and directly. Provide step-by-step instructions if applicable. Do not provide explanations about where the information came from. Do not ask the user any questions. 

**Your Task:**
1.  Analyze the User Question.
2.  Locate the relevant information *only* from the Knowledge Base.
3.  Answer the question directly and helpfully. If the Knowledge Base provides steps, list them clearly.

**Critical Rules:**
* **DO NOT** mention the Knowledge Base. Answer as if this is your own expert knowledge.
* **DO NOT** use any phrases like "According to the context," "The provided text states," or "Based on the information...".
* If the Knowledge Base does **not** contain the answer to the question, you must state that you cannot provide that specific information. Do not invent an answer.

**Knowledge Base:**
---
{context}
---
"""

In [22]:
llm = Llama.from_pretrained(
	repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
	filename="gemma-3-4b-it-q4_0.gguf",
 	n_ctx=4096,       
 	n_gpu_layers=-1,   
 	verbose=False     
)

print(f"Model loaded successfully.")

./gemma-3-4b-it-q4_0.gguf:   0%|          | 0.00/3.16G [00:00<?, ?B/s]

llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified_iswa: using full-size SWA cache (ref: https://github.com/ggml-org/llama.cpp/pull/13194#issuecomment-2868343055)


Model loaded successfully.


In [35]:
def chatbot(user_query: str, max_tokens:int=512):  
    retrieved_chunks = retriever.retrieve(user_query, top_k=5)
    reranked_chunks, scores = reranker.rerank(user_query, retrieved_chunks)
    context = "\n\n".join(reranked_chunks)
    
    system_prompt = PROMPT_TEMPLATE.format(context=context)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    try:
        output_generator = llm.create_chat_completion(
            messages=messages,
            max_tokens=384,
            temperature=0.9,
            top_p=0.95,
            stream=True  
        )
        
        for chunk in output_generator:
            delta = chunk["choices"][0]["delta"]
            content = delta.get("content")
            
            if content:
                yield content  

    except Exception as e:
        print("Model generation error:", str(e))
        yield "Sorry, something went wrong during response generation."

In [24]:
query = "what are the different types of screw available"

print(f"User: {query}")
print("Chatbot: ", end="", flush=True)

for text_chunk in chatbot(query):
    print(text_chunk, end="", flush=True)

print()

User: what are the different types of screw available
Chatbot: Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

There are several types of screws available. Mini eyeglasses, mobile phones, computers, and personal electronics often have very small screws, which in turn will require very small screwdrivers. These come in flat-head, phillips, torx™, pozidriv, and more.


In [30]:
query = "how to build a toolbox"

print(f"User: {query}")
print("Chatbot: ", end="", flush=True)

for text_chunk in chatbot(query,1024):
    print(text_chunk, end="", flush=True)

print()

User: how to build a toolbox
Chatbot: Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You can't build a toolbox from scratch using the information provided. However, you can certainly build a toolbox to *hold* your tools. Here’s how to create a basic, functional toolbox:

**Materials You’ll Need:**

*   Plywood (¾ inch thick is a good choice)
*   Wood glue
*   Screws (1 ½ inch)
*   Sandpaper (various grits)
*   Wood finish (paint, stain, or varnish – optional)

**Tools You’ll Need:**

*   Circular saw or hand saw
*   Drill
*   Screwdriver
*   Measuring tape
*   Square
*   Clamps

**Step-by-Step Instructions:**

1.  **Cut the Plywood:** Cut the plywood into the following pieces:
    *   **Top and Bottom:** Two pieces, approximately 12” x 18” (adjust to your desired size).
    *   **Sides:** Two pieces, approximately 12” x 8” (adjust to desired size).
    *   **Back:** One piece, approximately 12” x 8”.
    *   **Dividers (Optional):** These will create compartments within your toolbox. Cut pieces of plywood to the desired height and width for each compartment.

2.  **Ass

In [26]:
query = "list some of the big diy projects i can make in weekend"

print(f"User: {query}")
print("Chatbot: ", end="", flush=True)

for text_chunk in chatbot(query):
    print(text_chunk, end="", flush=True)

print()

User: list some of the big diy projects i can make in weekend
Chatbot: Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You can tackle a wide range of projects in a weekend with the right preparation. Here are some big DIY projects you could realistically complete:

*   **Crown Molding Installation:** You’ll be able to line your living room and hallway with beautiful crown molding.
*   **Chair Rail Installation:** Combining this with crown molding creates a more elaborate look.
*   **Painting:** A significant room or multiple smaller rooms can be painted effectively.
*   **Replacing Doors & Windows:** You can replace standard doors and windows.
*   **Pouring a Concrete Slab:** While it requires some effort, a small concrete slab is achievable.
*   **Installing Gutters and Downspouts:** This is a substantial outdoor project.


In [27]:
query = "how to fix leakages in pipes"

print(f"User: {query}")
print("Chatbot: ", end="", flush=True)

for text_chunk in chatbot(query):
    print(text_chunk, end="", flush=True)

print()

User: how to fix leakages in pipes
Chatbot: Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

To fix leakages in pipes, you can use self-fusing silicone tape. Stretch a length of the tape and wrap it around the problem area, overlapping the tape and making sure to cover the surface on both sides of the leak. The tape will fuse to itself to form a waterproof seal.


In [28]:
query = "what are the improvements i can make in my kitchen"

print(f"User: {query}")
print("Chatbot: ", end="", flush=True)

for text_chunk in chatbot(query,512):
    print(text_chunk, end="", flush=True)

print()

User: what are the improvements i can make in my kitchen
Chatbot: Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You can make several improvements to your kitchen, focusing on lighting, finishes, and potentially storage.

Here’s a breakdown of what you can do:

*   **Lighting:** Consider adding under-cabinet lighting to provide focused illumination for tasks like food preparation. You could also add pendant lighting over an island or countertop area. Combining different lighting types – like recessed lighting for general illumination and task lighting for specific areas – will create a more balanced and functional space.

*   **Cabinet Finishes:** Over time, kitchen cabinets can look worn. A fresh coat of paint can dramatically revitalize them. You can choose from a variety of finishes like wrought iron, pewter, satin nickel, or painted finishes. Consider colored glass for an updated look.

*   **Storage:** If you have limited space, look for ways to optimize storage. You could convert the space under a staircase into a closet, build storage bins under a deck, or add a garden shed for lawn tools.

In [37]:
!zip -r /kaggle/working/vectordb.zip /kaggle/working/vectordb

  adding: kaggle/working/vectordb/ (stored 0%)
  adding: kaggle/working/vectordb/collection/ (stored 0%)
  adding: kaggle/working/vectordb/collection/KnowDIY_DB/ (stored 0%)
  adding: kaggle/working/vectordb/collection/KnowDIY_DB/storage.sqlite

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 53%)
  adding: kaggle/working/vectordb/.lock (stored 0%)
  adding: kaggle/working/vectordb/meta.json (deflated 56%)


In [41]:
def gradio_streaming_chatbot(user_query):
    response = ""
    for chunk in chatbot(user_query.lower()):  
        response += chunk
        yield response  

with gr.Blocks() as demo:
    gr.Markdown("## 🧰 DIY Expert Assistant")

    user_input = gr.Textbox(
        label="Ask your DIY question:",
        placeholder="e.g., How to fix a leaking pipe?",
        lines=2
    )
    output_box = gr.Textbox(
        label="Response",
        placeholder="Answer will appear here...",
        lines=15
    )
    submit_btn = gr.Button("Ask")

    submit_btn.click(
        fn=gradio_streaming_chatbot,
        inputs=user_input,
        outputs=output_box
    )

demo.queue()  
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://bdfa838ecc945f8d34.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings for 1 texts ... 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 768)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]